# Custom Map File Creation notebook

This notebook will develop code to create the files that are necessary in the Beat Saber custom songs directory:

- .dat files (info.dat, 'level'.dat)
- cover.jpg file
- song.egg file

In [13]:
import numpy as np
import pandas as pd
import librosa
import json
import requests
import pickle
import matplotlib.pyplot as plt
from io import BytesIO, TextIOWrapper, StringIO
from zipfile import ZipFile
import os
import soundfile as sf
import audioread
from pydub import AudioSegment

In [2]:
#{'_version': '2.0.0',
#  '_songName': 'Gamle Kjente_V03',
#  '_songSubName': 'Despacito',
#  '_songAuthorName': 'Vegard Bakkely',
#  '_levelAuthorName': 'Martin Ask Eriksen',
#  '_beatsPerMinute': 89,
#  '_songTimeOffset': 0,
#  '_shuffle': 0,
#  '_shufflePeriod': 0.5,
#  '_previewStartTime': 6,
#  '_previewDuration': 40,
#  '_songFilename': 'song.egg',
#  '_coverImageFilename': 'cover.jpg',
#  '_environmentName': 'DefaultEnvironment',
#  '_customData': {'_editor': 'beatmapper',
#   '_editorSettings': {'enabledFastWalls': False, 'modSettings': {}}},
#  '_difficultyBeatmapSets': [{'_beatmapCharacteristicName': 'Standard',
#    '_difficultyBeatmaps': [{'_difficulty': 'Easy',
#      '_difficultyRank': 1,
#      '_beatmapFilename': 'Easy.dat',
#      '_noteJumpMovementSpeed': 10,
#      '_noteJumpStartBeatOffset': 0,
#      '_customData': {'_editorOffset': 0, '_requirements': []}}]}]}

def write_info(song_name, bpm, difficulty):
    """This function creates the 'info.dat' file that needs to be included in the custom folder."""

    difficulty_rank = None
    jump_movement = None
    if difficulty.casefold() == 'easy'.casefold():
        difficulty_rank = 1
        jump_movement = 8
    elif difficulty.casefold() == 'normal'.casefold():
        difficulty_rank = 3
        jump_movement = 10
    elif difficulty.casefold() == 'hard'.casefold():
        difficulty_rank == 5
        jump_movement = 12
    elif difficulty.casefold() == 'expert'.casefold():
        difficulty_rank == 7
        jump_movement = 14
    elif difficulty.casefold() == 'expertPlus'.casefold():
        difficulty_rank == 9
        jump_movement = 16
            
    info = {'_version': '2.0.0',
            '_songName': f"{song_name}",
            '_songSubName': '',
            '_songAuthorName': '',
            '_levelAuthorName': 'BeatMapSynth',
            '_beatsPerMinute': round(bpm),
            '_songTimeOffset': 0,
            '_shuffle': 0,
            '_shufflePeriod': 0,
            '_previewStartTime': 10,
            '_previewDuration': 30,
            '_songFilename': 'song.egg',
            '_coverImageFilename': 'cover.jpg',
            '_environmentName': 'DefaultEnvironment',
            '_customData': {}, #I don't think anything is needed here
             '_difficultyBeatmapSets': [{'_beatmapCharacteristicName': 'Standard',
                                         '_difficultyBeatmaps': [{'_difficulty': f"{difficulty}",
                                                                  '_difficultyRank': difficulty_rank,
                                                                  '_beatmapFilename': f"{difficulty}.dat",
                                                                  '_noteJumpMovementSpeed': jump_movement, #not sure what this is, seems to vary with difficulty level, may want to 
                                                                  '_noteJumpStartBeatOffset': 0, #ditto
                                                                  '_customData': {}}]}]} #{'_editorOffset': 0, '_requirements': []}}]}]} - don't think this is needed
    with open('info.dat', 'w') as f:
        json.dump(info, f)


In [3]:
# {'_version': '2.0.0',
#  '_customData': {'_time': 220, '_BPMChanges': [], '_bookmarks': []},
#  '_events': [{'_time': 0, '_type': 1, '_value': 3}, LIST],
#  '_notes': [{'_time': 12.5,
#    '_lineIndex': 1,
#    '_lineLayer': 0,
#    '_type': 0,
#    '_cutDirection': 1}, LIST],
#  '_obstacles': [{'_time': 0,
#    '_lineIndex': 3,
#    '_type': 0,
#    '_duration': 9,
#    '_width': 1}, LIST]}

def write_level(difficulty, events_list, notes_list, obstacles_list):
    """This function creates the 'level.dat' file that contains all the data for that paticular difficulty level"""
    
    level = {'_version': '2.0.0',
             '_customData': {'_time': '', #not sure what time refers to 
                             '_BPMChanges': [], 
                             '_bookmarks': []},
             '_events': events_list,
             '_notes': notes_list,
             '_obstacles': obstacles_list}
    with open(f"{difficulty}.dat", 'w') as f:
        json.dump(level, f)

In [4]:
def beat_features(song_path):
    """This function takes in the song stored at 'song_path' and estimates the bpm and beat times."""
    #Load song and split into harmonic and percussive parts.
    y, sr = librosa.load(song_path)
    y_harmonic, y_percussive = librosa.effects.hpss(y)
    #Isolate beats and beat times
    bpm, beat_frames = librosa.beat.beat_track(y=y_percussive, sr=sr)
    beat_times = librosa.frames_to_time(beat_frames, sr=sr)
    return bpm, beat_times

In [5]:
def music_file_converter(song_path):
    """This function makes sure the file type of the provided song will be converted to the music file type that 
    Beat Saber accepts"""
    if song_path.endswith('.mp3'):
        AudioSegment.from_mp3(song_path).export('song.egg', format='ogg')
    elif song_path.endswith('.wav'):
        AudioSegment.from_wav(song_path).export('song.egg', format='ogg')
    elif song_path.endswith('.flv'):
        AudioSegment.from_flv(song_path).export('song.egg', format='ogg')
    elif song_path.endswith('.raw'):
        AudioSegment.from_raw(song_path).export('song.egg', format='ogg')
    elif song_path.endswith('.ogg') or song_path.endswith('.egg'):
        pass
    else:
        print("Unsupported song file type. Choose a file of type .mp3, .wav, .flv, .raw, or .ogg.")


In [6]:
def random_notes_writer(beat_times, difficulty):
    """This function randomly places blocks at approximately each beat or every other beat depending on the difficulty."""
    notes_list = []
    line_index = [0, 1, 2, 3]
    line_layer = [0, 1, 2]
    types = [0, 1, 2, 3]
    directions = list(range(0, 10))
    beat_times = [float(x) for x in beat_times]
    
    if difficulty == 'Easy' or difficulty == 'Normal':
        for beat in beat_times:
            empty = np.random.choice([0,1])
            if empty == 1:
                note = {'_time': beat,
                        '_lineIndex': int(np.random.choice(line_index)),
                        '_lineLayer': int(np.random.choice(line_layer)),
                        '_type': int(np.random.choice(types)),
                        '_cutDirection': int(np.random.choice(directions))}
                notes_list.append(note)
            else:
                continue
    else:
        random_beats = np.random.choice(beat_times, np.random.choice(range(len(beat_times)))) #randomly choose beats to have more than one note placed
        randomly_duplicated_beat_times = np.concatenate([beat_times, random_beats])
        randomly_duplicated_beat_times.sort()
        randomly_duplicated_beat_times = [float(x) for x in randomly_duplicated_beat_times]
        for beat in randomly_duplicated_beat_times:
            note = {'_time': beat,
                    '_lineIndex': int(np.random.choice(line_index)),
                    '_lineLayer': int(np.random.choice(line_layer)),
                    '_type': int(np.random.choice(types)),
                    '_cutDirection': int(np.random.choice(directions))}
            notes_list.append(note)
    return notes_list

In [7]:
def random_notes_writer_v2(beat_times, difficulty):
    """This function randomly places blocks at approximately each beat or every other beat depending on the difficulty."""
    notes_list = []
    line_index = [0, 1, 2, 3]
    line_layer = [0, 1, 2]
    types = [0, 1, 2, 3]
    directions = list(range(0, 10))
    #beat_times = [float(x) for x in beat_times]
    beat_times = list(range(len(beat_times)))
    
    if difficulty == 'Easy' or difficulty == 'Normal':
        for beat in beat_times:
            empty = np.random.choice([0,1])
            if empty == 1:
                note = {'_time': beat,
                        '_lineIndex': int(np.random.choice(line_index)),
                        '_lineLayer': int(np.random.choice(line_layer)),
                        '_type': int(np.random.choice(types)),
                        '_cutDirection': int(np.random.choice(directions))}
                notes_list.append(note)
            else:
                continue
    else:
        random_beats = np.random.choice(beat_times, np.random.choice(range(len(beat_times)))) #randomly choose beats to have more than one note placed
        randomly_duplicated_beat_times = np.concatenate([beat_times, random_beats])
        randomly_duplicated_beat_times.sort()
        randomly_duplicated_beat_times = [float(x) for x in randomly_duplicated_beat_times]
        for beat in randomly_duplicated_beat_times:
            note = {'_time': beat,
                    '_lineIndex': int(np.random.choice(line_index)),
                    '_lineLayer': int(np.random.choice(line_layer)),
                    '_type': int(np.random.choice(types)),
                    '_cutDirection': int(np.random.choice(directions))}
            notes_list.append(note)
    return notes_list

In [8]:
def events_writer(beat_times):
    
    events_list = []
    return events_list

In [9]:
def obstacles_writer(beat_times, difficulty):
    
    obstacles_list = []
    return obstacles_list

In [10]:
def zip_folder_exporter(song_name, difficulty):
    "This function exports the zip folder containing the info.dat, difficulty.dat, cover.jpg, and song.egg files."
    files = ['info.dat', f"{difficulty}.dat", 'cover.jpg', 'song.egg']
    with ZipFile(f"{song_name}.zip", 'w') as custom:
        for file in files:
            custom.write(file)

In [11]:
def random_mapper(song_path, song_name, difficulty):
    """Function to output the automatically created completely random map (i.e., baseline model) for a provided song.
    Returns a zipped folder that can be unzipped and placed in the 'CustomMusic' folder in the Beat Saber game
    directory and played. CAUTION: This is completely random and is likely not enjoyable if even playable!"""
    #Load song and get beat features
    print("Loading Song...")
    bpm, beat_times = beat_features(song_path)
    print("Song loaded successfully!")
    #Write lists for note placement, event placement, and obstacle placement
    print("Random mapping...")
    #notes_list = random_notes_writer(beat_times, difficulty) 
    notes_list = random_notes_writer_v2(beat_times, difficulty) #fixes _time != beat time
    events_list = events_writer(beat_times)
    obstacles_list = obstacles_writer(beat_times, difficulty)
    print("Mapping done!")
    #Write and zip files
    print("Writing files to disk...")
    write_info(song_name, bpm, difficulty)
    write_level(difficulty, events_list, notes_list, obstacles_list)
    print("Converting music file...")
    music_file_converter(song_path)
    print("Zipping folder...")
    zip_folder_exporter(song_name, difficulty)
    print("Finished! Look for zipped folder in your current path, unzip the folder, and place in the 'CustomMusic' folder in the Beat Saber directory")

In [22]:
random_mapper('song.egg', 'example', 'expert')

Loading Song...


/opt/anaconda3/envs/BSmapsynth-env/lib/python3.6/site-packages/librosa/core/audio.py:146: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed. Trying audioread instead.')


Song loaded successfully!
Random mapping...
Mapping done!
Writing files to disk...
Converting music file...
Zipping folder...
Finished! Look for zipped folder in your current path, unzip the folder, and place in the 'CustomMusic' folder in the Beat Saber directory


In [12]:
random_mapper('Sro_-_Submersed_Phonics_JBYo_Collab.mp3', 'Sro_example', 'Normal')

Loading Song...


/opt/anaconda3/envs/BSmapsynth-env/lib/python3.6/site-packages/librosa/core/audio.py:146: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed. Trying audioread instead.')


Song loaded successfully!
Random mapping...
Mapping done!
Writing files to disk...
Converting music file...


CouldntEncodeError: Encoding failed. ffmpeg/avlib returned error code: 1

Command:['ffmpeg', '-y', '-f', 'wav', '-i', '/var/folders/zx/4lkw7gdj4s9b5t2lt28dm2lm0000gp/T/tmp6ikn0t8u', '-acodec', 'libvorbis', '-f', 'ogg', '/var/folders/zx/4lkw7gdj4s9b5t2lt28dm2lm0000gp/T/tmp9dg2ot7e']

Output from ffmpeg/avlib:

b"ffmpeg version 4.2 Copyright (c) 2000-2019 the FFmpeg developers\n  built with clang version 4.0.1 (tags/RELEASE_401/final)\n  configuration: --prefix=/opt/anaconda3/envs/BSMapSynth-env --cc=x86_64-apple-darwin13.4.0-clang --disable-doc --enable-gpl --enable-avresample --disable-gnutls --enable-hardcoded-tables --enable-libfreetype --enable-openssl --enable-libvpx --enable-pthreads --enable-libopus --enable-postproc --enable-pic --enable-pthreads --enable-shared --enable-static --enable-version3 --enable-zlib --enable-libmp3lame --enable-nonfree --enable-libopenh264 --enable-libx264\n  libavutil      56. 31.100 / 56. 31.100\n  libavcodec     58. 54.100 / 58. 54.100\n  libavforma

In [14]:
music_file_converter('Sro_-_Submersed_Phonics_JBYo_Collab.mp3')